In [1]:
%matplotlib inline  
import matplotlib.pyplot as plt   
import pandas as pd
import random
import numpy as np
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook
from time import sleep
import scipy
import operator
import difflib
import math

from IPython.core.display import display,HTML
display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser



try:
    import cPickle as pickle  ##################################
   #it is faster than pickle!
except:
    import pickle
    
import unicodedata
import networkx as nx
import itertools
import seaborn as sns   ### https://seaborn.pydata.org/tutorial/categorical.html
import time  



import plotly.plotly as py


# i only need my credentials if i want to plot online --- and send plots to server (limits per day apply!)
#import plotly.tools as tls
#tls.set_credentials_file(username='juliettapc', api_key='deyNIvtOoDZ5PLmrHlhd')  # my plotly account credentials


import pygraphviz
from networkx.drawing.nx_agraph import graphviz_layout




########## to be able to plot offline (without sending the plots to the plotly server every time)
import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
################



##### for getting geolocation data  and to calculate distance between two geolocations
import requests
import json
import geopy.distance   




In [ ]:
# path = "/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Dropbox/"
# input_file = 'Dropbox_datafile_complete_new_ranking.csv'
# #input_file = 'Dropbox_datafile_may22_2017_modified_added_univ_country_geolocation_RUCC_population_density_when_available_career_stage_GINI_folder_act_categories.csv'
# df_old = pd.read_csv(path+input_file, sep=';',na_values=["NAN","-1","null"],low_memory=False, parse_dates=['folder_creation_date','date_last_change']) # set header=0 if i wanna pass it my own list of header names
# df_old.drop('Unnamed: 0', axis=1, inplace=True)
# print df_old.shape   #  1403349, 38 



# path = "/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Dropbox/"
# input_file = 'ORIGINAL_Dropbox_datafile_may22_2017.csv'
# #input_file = 'Dropbox_datafile_may22_2017_modified_added_univ_country_geolocation_RUCC_population_density_when_available_career_stage_GINI_folder_act_categories.csv'
# df_old_old = pd.read_csv(path+input_file, sep=',',na_values=["NAN","-1","null"],low_memory=False)#, parse_dates=['folder_creation_date','date_last_change']) # set header=0 if i wanna pass it my own list of header names
# print df_old_old.shape  


# ######## load dictionary aggregated info by folder
# pickle_name='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_folder_id_folder_attr_even_more_var_.pickle'
# with open(pickle_name, 'rb') as handle:
#     dict_folder_id_folder_attr = pickle.load(handle)
 
# print len(dict_folder_id_folder_attr.keys())
# # df_folders = pd.DataFrame.from_dict(dict_folder_id_folder_attr,orient='index')
# # print df_folders.shape   # 521274, 13





# ######## load dictionary aggregated info by user
# pickle_name='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_user_id_user_attr_more_var_.pickle'
# with open(pickle_name, 'rb') as handle:
#     dict_user_id_user_attr = pickle.load(handle)
# print len(dict_user_id_user_attr.keys())

# # df_users = pd.DataFrame.from_dict(dict_user_id_user_attr,orient='index')
# # #df_selection_users=df_folders[df_folders['user_univ_ranking']<=50]
# # print df_users.shape   # 440353, 11





# ############  load the aggregated network
# pickle_name='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/Networks/network_all_new.pickle'
# file = open(pickle_name,'r')
# G_no_filter = pickle.load(file)
# print "\n",pickle_name, len (G_no_filter.nodes()), len(G_no_filter.edges())    #   440353 3659098




# ############  load the aggregated network   filtering links by activity (if a node's activity =0, then it is not link to any other members of a folder)
# pickle_name='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/Networks/network_all_weighted_by_act_thresh1.pickle'
# file = open(pickle_name,'r')
# G_activity = pickle.load(file)
# print "\n",pickle_name, len (G_activity.nodes()), len(G_activity.edges())    #   440353 3659098







In [ ]:
# print df_old_old.shape 

In [ ]:
# lista_users_old=list(df_old.user_id.unique())
# lista_users_old_old=list(df_old_old.member_id.unique())

In [ ]:
# df[df['user_id']==1842650] 

In [ ]:
# df_old[df_old['user_id']==1842650]#.head(1000)

In [3]:


# header_names = ['folder_id','num_members','folder_creation_date','last_date_DO_NOT_TRUST','user_id','email_domain','num_adds','num_edits','num_dels',\
#                 'major_content_type','major_file_ext','date','field','new_group_total_pubs','new_group_num_papers_last','new_group_num_citations']
    
    
# header_names = ['folder_id','num_members','folder_creation_date','last_date_DO_NOT_TRUST','user_id','email_domain','num_adds','num_edits','num_dels',\
#                 'major_content_type','major_file_ext','date','field','new_group_total_pubs','new_group_num_papers_last','new_group_num_citations',\
#                 'simplified_domain','University_Name','Cleaned_university_name','Country']
 
header_names = ['unnamed', 'folder_id','num_members','folder_creation_date','user_id','num_adds','num_edits','num_dels','date','field',\
                'new_group_total_pubs','new_group_num_papers_last','new_group_num_citations','Cleaned_university_name','Country']



print "reading file......."

path = "/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Dropbox/"
input_file = 'new_DROPBOX_add_univ_names_country.csv'    #   'new_DROPBOX.csv'   #     new_sorted_no_duplicates.csv'   #ORIGINAL_Nico_Results_Hashed_RH

df_user = pd.read_csv(path+input_file, sep=',',na_values=["NAN","-1","null"],low_memory=False, usecols=[4,9,10,11,12,13,14],  parse_dates=['folder_creation_date','date']) # usecols=[0,1,2,4,5,6,7,8,11,12,13,14,15], set header=0 if i wanna pass it my own list of header names
#df.drop('unnamed', axis=1, inplace=True)

print df_user.shape   #  (9018863, 16) 
df_user=df.drop_duplicates()  # i have dropped them from the shell already
print df_user.shape , "after dropping duplicates"




new_name=path+"df_new_dropbox_users.csv"
df_user.to_csv(new_name, sep=',')
print "written:", new_name,"\n\n\n"


reading file.......


CParserError: Error tokenizing data. C error: out of memory

In [ ]:
print len(df.folder_id.unique()) , "unique folders"#  345,043 users,     417,657  folders
print len(df.user_id.unique()) , "unique users"

In [ ]:
df_complete_info=df[['folder_id','user_id','email','new_group_num_citations']].dropna(how='any')  # by default, dropna drops a row if ANY  of the indicated fields are missing (i can also set it to if ALL the fields are missing: how='all')
print df_complete_info.shape

In [ ]:
df.folder_creation_date.max()    # min date: 2015-05-15 , max: 2017-05-15       ## folder creation date, min: 2008-05-01,  max: 2017-05-16 

In [ ]:
df.columns


In [ ]:
df[df['user_id'] ==881645].sort_values(by=['folder_id','date'])

In [ ]:

#to get rid of the cumulative countings and redundant rows without new info, and get actual new activity counts for each date

def  grouping(input_df):

   
    new_var=np.nan 
    user_id =  input_df.user_id.iloc[0]
    folder_id= input_df.folder_id.iloc[0]
    new_var= str(user_id) +"_" +str(folder_id)
    
    
    
    list_values_add=[]   # i need to create a list of actual activity values, not cumulative ones
    list_values_edit=[]
    list_values_del=[]    
        
   
        
                
    list_indeces=[]
    flag_new=1

    old_tupla_activity=(None,None,None)        

    old_value_add=0
    old_value_edit=0
    old_value_del=0
    old_date=pd.Timestamp('1982-01-01')  # arbitrarily old date
      
    
    cont=1
    for row in input_df.iterrows():
        index=row[0]

        num_adds=row[1].num_adds_last28days
        num_edits=row[1].num_edits_last28days
        num_dels=row[1].num_dels_last28days

        tupla_activity=(num_adds,num_edits ,num_dels)  #  num_adds_last28days','num_edits_last28days','num_dels_last28days


        date=row[1].date_act
        try:
            print old_date, date, (date-old_date).days
            raw_input()
        except:
            print date, "no date, no problem"
        
        if tupla_activity == old_tupla_activity:
            flag_new =0
        else:
            flag_new=1


            

        if flag_new ==1:   # OJO!!!  double check! THE RESETTING TO TAKE INTO ACCOUNT THE 28 DAY PERIOD!!!!!!!!11  


            list_indeces.append(index)


            if old_value_add  <=   num_adds: 
                actual_val_add = num_adds - old_value_add
            else:
                actual_val_add = num_adds                                                                         

            if old_value_edit <= num_edits:       
                actual_val_edits = num_edits- old_value_edit
            else:
                actual_val_edits = num_edits

            if old_value_del  <=  num_dels:
                actual_val_del = num_dels - old_value_del                
            else:
                actual_val_del = num_dels 



            list_values_add.append(actual_val_add)
            list_values_edit.append(actual_val_edits)
            list_values_del.append(actual_val_del)


            

        old_tupla_activity=tupla_activity


        old_value_add= num_adds
        old_value_edit= num_edits
        old_value_del= num_dels


        cont +=1

        selected_df=input_df.ix[list_indeces]


        selected_df['num_adds'] = pd.Series(list_values_add, index=list_indeces)
        selected_df['num_edits'] = pd.Series(list_values_edit, index=list_indeces)
        selected_df['num_dels'] = pd.Series(list_values_del, index=list_indeces)
    
 

    return selected_df
    
    

    
    
 
##########################

    
df_filtered = pd.DataFrame(columns=header_names)
cont_tot=0
cont_selection=0
for user_id in tqdm_notebook(df['user_id'].head(50000).unique()):
        
    df_selection_user= df[df['user_id'] == user_id]
    
    for folder_id in df_selection_user['folder_id'].unique():
        
        df_sub_selection =  df_selection_user[df_selection_user['folder_id'] == folder_id].sort_values(by='date')
                                          
        cont_tot += len(df_sub_selection)
        
   

        df_result= grouping(df_sub_selection)
        df_filtered = pd.concat([df_filtered,df_result])
        
        cont_selection += len(df_result)

        
        
        
print "done"  # 667133998     18092141 (many rows!)    298373200    87241372    205282877   205308895


In [ ]:
print  df.shape, df_filtered.shape

In [ ]:
path = "/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Dropbox/"
input_file = 'new_sorted_no_duplicates.csv'
new_file_name=path+input_file.replace(".csv","")+"_processed_timestaps_PARTIAL.csv"
df_filtered.to_csv(new_file_name, sep=',')
print "written:", new_file_name,"\n\n\n"

In [ ]:
print cont_tot, cont_selection   #one row: 130515361    3327476   108763580


In [ ]:
df[df['user_id']==34049545].sort_values(by='date')  # 273831436    383267705  667133998  18092141

In [ ]:
df_filtered[df_filtered['user_id']==34049545].sort_values(by='date')

In [ ]:
18092141

In [ ]:
df[df['user_id']==273831436].sort_values(by='date')  #.shape

In [ ]:
df_new[df_new['user_id']==96751].sort_values(by='date')
#df_new.head()

In [ ]:
# lista_users=list(df.user_id.unique())
# set(lista_users)  & set(lista_users_old_old)

In [ ]:
#  df=pandas.read_csv(path+filename, sep=',',names = ['First_Name','Last_Name','Title','Email','University'-,'School_college','Departments','Url'])
    
#     usecols=["date", "loc", "x"]
    
# #     df.rename(columns={'old_col':'new_col', 'old_col_2':'new_col_2'}, inplace=True)
    
# #     df.rename(columns={'old_col':'folder_id', 'old_col':'num_members', 'old_col':'folder_creation_date', 'old_col':'last_date_DO_NOT_TRUST', 'old_col':'user_id',\
# #                        'old_col':'email', 'old_col':'num_adds_last28days', 'old_col':'num_edits_last28days', 'old_col_2':'num_dels_last28days', \
# #                        'old_col_2':'new_col_2', 'old_col_2':'new_col_2', 'old_col_2':'new_col_2', 'old_col_2':'new_col_2', 'old_col_2':'new_col_2', 'old_col_2':'new_col_2', 'old_col_2':'new_col_2', 'old_col_2':'new_col_2', 'old_col_2':'new_col_2'}, inplace=True)
     
    
# names = ['folder_id','num_members','folder_creation_date','last_date_DO_NOT_TRUST','user_id','email','num_adds_last28days','num_edits_last28days','num_dels_last28days',\
#     'major_content_type','major_file_ext','date','field','new_group_total_pubs','new_group_num_papers_last','new_group_num_citations']
    
    
    
#     If you only want to read the first 999,999 (non-header) rows:

# read_csv(..., nrows=999999)
# If you only want to read rows 1,000,000 ... 1,999,999

# read_csv(..., skiprows=1000000, nrows=999999)

# FALTA DROP DUPLICATES!   df=df.drop_duplicates()